# IHLT final project - Semantic Textual Similarity
Nikita Belooussov and Santiago del Rey Juárez

## Introduction

In [1]:
import csv
import os
import pandas as pd
import numpy as np
import nltk
from nltk.metrics import jaccard_distance
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from nltk.stem import WordNetLemmatizer
import string
import spacy
from nltk.metrics import jaccard_distance

nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('conll2000')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet_ic')
nltk.download('wordnet')

nlp = spacy.load("en_core_web_sm") #if this does not work run python -m spacy download en in terminal and restart the program running the code

brown_ic = wordnet_ic.ic('ic-brown.dat')

!pip install svgling

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nbelo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nbelo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\nbelo\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\nbelo\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nbelo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nbelo\AppData\Roaming\nltk_data...
[nltk_data]   Pac

In [ ]:
def read_file(file_path):
    return pd.read_csv(file_path, sep='\t', lineterminator='\n', names=['sentence0', 'sentence1'], header=None,
                       quoting=csv.QUOTE_NONE)

Lemmatize

In [ ]:
from nltk.corpus import wordnet

wnl = nltk.stem.WordNetLemmatizer()


def lemmatize(p):
    if p[1][0] in {'N', 'V', 'J', 'R'}:  #N- noun, V- verb, J- adjective, R-adverb
        if p[1][0] is 'J':  #this is used due to wordnet using a different label for adjectives than one given by nltk
            return wnl.lemmatize(p[0].lower(), pos=wordnet.ADJ)
        return wnl.lemmatize(p[0].lower(), pos=p[1][0].lower())
    return p[0]

<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\santi\AppData\Local\Temp/ipykernel_39864/1815987616.py:7: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if p[1][0] is 'J':  #this is used due to wordnet using a different label for adjectives than one given by nltk


Computes the Jaccard Distance 

In [ ]:
punct = string.punctuation


def remove_punct_and_lower(s0, s1):
    s0 = [w.lower() for w in s0 if not all(c in punct for c in w)]
    s1 = [w.lower() for w in s1 if not all(c in punct for c in w)]
    return s0, s1

computed_synsets = {}

def get_wordnet_similarity(s0, s1, method):
    if method == "path" and s0 is not None and s1 is not None:
        return s0.path_similarity(s1)
    elif method == "lch" and s0 is not None and s1 is not None and s0.pos == s1.pos:
        return s0.lch_similarity(s1)
    elif method == "wup" and s0 is not None and s1 is not None:
        return s0.wup_similarity(s1)
    elif method == "lin" and s0 is not None and s1 is not None and s1.pos == s1.pos and s0.pos in {'n', 'v', 'r', 'a'}:
        return s0.lin_similarity(s1)
    else:
        return None

def max_similarity(s0, s1, method):
    if s0 == s1:
        return 1

    if (s0, s1, method) in computed_synsets:
        return computed_synsets[(s0, s1, method)]

    synsets0 = wordnet.synsets(s0)
    synsets1 = wordnet.synsets(s1)

    similarities = []
    for syn0 in synsets0:
        for syn1 in synsets1:
            similarity = get_wordnet_similarity(syn0, syn1, method)
            if similarity is not None:
                similarities.append(similarity)

    if len(similarities) > 0:
        max_sim = max(similarities)
        if method == 'lch':
            # If Leacock similarity divide by 3 in order to normalize the similarity
            computed_synsets[(s0, s1, method)] = max_sim/3
            return max_sim/3
        else:
            computed_synsets[(s0, s1, method)] = max_sim
            return max_sim
    else:
        computed_synsets[(s0, s1, method)] = 0
        return 0

def synset_similarity(lemmas0, lemmas1, method):
    # TODO
    return None

def compute_jaccard(sentence_0, sentence_1, remove_punctuation=True, type='nltk'):
    s0 = nltk.word_tokenize(sentence_0)
    s1 = nltk.word_tokenize(sentence_1)

    if remove_punctuation:
        s0, s1 = remove_punct_and_lower(s0, s1)

    if type == 'lemma':
        return lemmatize_sentences(s0, s1)

    elif type == "NE":
        s0_ne = name_entity_tokenization(sentence_0)

        s1_ne = name_entity_tokenization(sentence_1)

        return 1 - jaccard_distance(set(s0_ne), set(s1_ne))

    elif type == "Leacock" or type == "Lin":
        pairs_s0 = nltk.pos_tag(s0)
        pairs_s1 = nltk.pos_tag(s1)

        syn = []
        for pair in pairs_s0:
            try:
                syn.append(wn.synsets(pair[0], pair[1].lower()[0])[0])
            except:
                pass

        if type == "Leacock":
            lSimA = round(syn[0].lch_similarity(syn[0]), 3)
            try:
                lSim = round(word.lch_similarity(word2) / lSimA, 3)
            except:
                lSimA.append(0)
        if type == "Lin":
            liSimA = round(syn[0].lin_similarity(syn[0], brown_ic), 3)
            try:
                liSim = round(word.lin_similarity(word2, brown_ic) / liSimA, 3)
            except:
                liSimA.append(0)


def name_entity_tokenization(sentence):
    doc = nlp(sentence.lower())
    with doc.retokenize() as retokenizer:
        tokens = [token for token in doc]
        for ent in doc.ents:
            retokenizer.merge(doc[ent.start:ent.end],
                              attrs={"LEMMA": " ".join([tokens[i].text for i in range(ent.start, ent.end)])})
    s0_ne = [token.text for token in doc]
    return s0_ne


def lemmatize_sentences(s0, s1):
    pairs_s0 = nltk.pos_tag(s0)
    pairs_s1 = nltk.pos_tag(s1)
    s0 = [lemmatize(pair) for pair in pairs_s0]
    s1 = [lemmatize(pair) for pair in pairs_s1]
    return s0, s1

In [ ]:
path = os.path.join('data', 'train', 'STS.input.MSRpar.txt')

df_input = read_file(path)
df_input.head()

,sentence0,sentence1
0,But other sources close to the sale said Viven...,But other sources close to the sale said Viven...
1,Micron has declared its first quarterly profit...,Micron's numbers also marked the first quarter...
2,The fines are part of failed Republican effort...,"Perry said he backs the Senate's efforts, incl..."
3,"The American Anglican Council, which represent...","The American Anglican Council, which represent..."
4,The tech-loaded Nasdaq composite rose 20.96 po...,The technology-laced Nasdaq Composite Index <....


Random Forest Model

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 6, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)

SVR model

In [ ]:
from sklearn import svm
regr = svm.SVR()
regr.fit(X, y)
SVR()
regr.predict([[1, 1]])

Linear KNN

In [ ]:
model = neighbors.KNeighborsRegressor(n_neighbors = K)

model.fit(x_train, y_train)  #fit the model
pred=model.predict(x_test) #make prediction on test set